Sep 2025

This is a last step testing of WeatherMCP. I want to use it in a agent system that includes two MCP servers. The WeatherMCP server, and a memory server that can save a location related to a specific person. Then I will check if in a new session it can distinguish each person and load the location without recalling the api. But only when asked for any weather parameter it will use the related functions to retrieve information. I must design a good prompt here.

In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio

import os
from IPython.display import Markdown, display
from datetime import datetime

load_dotenv(override=True)

False

In [10]:
# Configure MCP servers
import os

# Get the project src directory
# In Jupyter notebooks, getcwd() returns the notebook directory
# If notebook is in notebooks/, src is ../src
project_root = os.path.dirname(os.getcwd())
project_src = os.path.join(project_root, 'src')

# Note: If the package is installed with 'pip install -e .', PYTHONPATH may not be needed
# but we include it here for cases where the package is not installed
params_weather_mcp = {
    "name": "weatherinfo-mcp",
    "command": "python",
    "args": ["-m", "weatherinfo_mcp.mcp_tools.main"],
    "env": {"PYTHONPATH": project_src}
}

params_memory_mcp = {
    "name": "memory-",
    "command": "npx",
    "args": ["-y", "mcp-memory-libsql"],
    "env": {"LIBSQL_URL": "file:./memory/test_memory.db"}
}


In [11]:
# Set up the agent with MCP servers

# Create MCPServerStdio for each MCP server
weather_server = MCPServerStdio(params=params_weather_mcp, client_session_timeout_seconds=30)
memory_server = MCPServerStdio(params=params_memory_mcp, client_session_timeout_seconds=30)
mcp_servers = [weather_server, memory_server]


instructions = """
You are a weather assistant with two specialized MCP tools:
1. A weather data tool for live weather information.
2. A memory management tool for storing and recalling user-related information.

Your responsibilities:
- Provide accurate and context-aware weather information.
- Maintain persistent memory of users and their associated locations.

Capabilities:
- Retrieve weather observations (temperature, humidity, wind, weather description).
- Report weather alerts only when the user explicitly asks for them.
- Provide relevant heat risk context as appropriate.
- Create, recall, and update user and location entities in long-term memory.

Memory and Identification Workflow:
1. Always begin by identifying the user. Use memory to check for an existing entity representing this person.
2. Use consistent entity naming when saving people, e.g.:
   person_<user_name>  →  location_<city_name>
3. If a person or location entity does not exist, create both:
   - person entity: represents the user (e.g. "John Doe")
   - location entity: represents their location (e.g. "New York, NY")
   - create_relation: link person to location using “lives_in”
4. After identifying the entities, use the stored location for all weather queries.

Weather Query Workflow:
1. Retrieve user’s stored location (if exists).
2. If not found, ask for it, then store using the memory tool.
3. Fetch new weather data using get_current_observation.
4. Extract only the requested details (e.g. temperature, humidity, or heat risk).
5. Only check and report active weather alerts if the user explicitly mentions alerts or warnings.

Example Interaction Flow:
User: "Hi, I’m John from New York. What’s the weather?"
- Retrieve or create person_John and location_NewYork entities.
- Ensure relation: person_John → lives_in → location_NewYork
- Fetch weather for New York and report observations clearly.

Response Guidelines:
- Only answer the question. Do need ask if the user may need anything else. Do not attempt to engage the user in a conversation.
- Never ask for a location again if it is already stored in memory.
- Persist all newly discovered user details consistently.
- Recognize returning users by their saved memory entities.
- Only mention weather alerts when explicitly requested.
- Always prioritize clarity, factual accuracy, and contextual completeness in responses.
"""




# Create agent with system prompt
agent = Agent(
    name="weather-agent",
    instructions=instructions,
    model="gpt-4.1-mini",
    mcp_servers=mcp_servers
)

print("✅ Weather agent created successfully!")
print("🤖 Agent has access to WeatherInfo_MCP weather tools")
print("💾 Memory functionality can be added later")


✅ Weather agent created successfully!
🤖 Agent has access to WeatherInfo_MCP weather tools
💾 Memory functionality can be added later


In [12]:
# Check what tools the agent has access to
print("🔧 Available tools for the agent:")
for server in agent.mcp_servers:
    print(f"\n📡 Server: {server.name}")
    try:
        # Connect each server before listing tools
        await server.connect()
        # This will show you what tools each MCP server provides
        tools = await server.list_tools()
        for tool in tools:
            print(f"  - {tool.name}: {tool.description}")
    except Exception as e:
        print(f"  ❌ Error getting tools: {e}")

🔧 Available tools for the agent:

📡 Server: stdio: python
  - create_location: 
    Create a Location object from an address or (latitude, longitude).

    Use this when the agent needs to work with a new place.

    Args:
        address: Human-readable address or place name.
        latitude: Latitude in decimal degrees.
        longitude: Longitude in decimal degrees.

    Returns:
        dict with Location information:
          - "address": The address (if provided)
          - "latitude": Latitude
          - "longitude": Longitude
          - "station_url": URL for nearest NWS observation station

    Notes:
        - The agent may save the returned dictionary for future queries about this location.
        - The station_url is resolved immediately during location creation for optimal performance.
    
  - get_current_observation: 
    Retrieve the latest weather observation from the location's nearest station.

    Args:
        location: Serialized Location dict
             

In [15]:
question = "Hi. My name is John Dove and I live in Omaha, Nebraska. What's the weather like today?"

for server in mcp_servers:
    await server.connect()
with trace("WeatherInfo_MCP_test"):
    result = await Runner.run(agent, question, max_turns=10)
display(Markdown(result.final_output))


The weather in Omaha, Nebraska, today is mostly clear with a temperature of around -10°C (14°F). The relative humidity is approximately 61.5%. There is a light wind coming from the southeast at about 7.4 km/h (4.6 mph). It feels a bit colder with a wind chill of about -14°C (6.9°F).

In [5]:
question = "Hi. I am Bobby Yribarren. Is there any weather alert today?"

for server in mcp_servers:
    await server.connect()
with trace("WeatherInfo_MCP_test"):
    result = await Runner.run(agent, question, max_turns=10)
display(Markdown(result.final_output))


I don't have your location saved yet. Could you please tell me the city or area where you are located? This will help me check for any weather alerts for your location.

In [16]:
question = "Hi. I am John Dove. Is there any weather alert today?"

for server in mcp_servers:
    await server.connect()
with trace("WeatherInfo_MCP_test"):
    result = await Runner.run(agent, question, max_turns=10)
display(Markdown(result.final_output))


John, there are no active weather alerts today for Omaha, Nebraska.

In [13]:
question = "Hi. My name is Babak Fard and I live in Boston, Massachusetts. What's the weather like today?"

for server in mcp_servers:
    await server.connect()
with trace("WeatherInfo_MCP_test"):
    result = await Runner.run(agent, question, max_turns=10)
display(Markdown(result.final_output))


Babak Fard, the current weather in Boston, Massachusetts is cloudy with a temperature of about 11°C. The wind is coming from the southeast at around 16.7 km/h with gusts up to 27.8 km/h.

In [14]:
question = "Hi. I am Babak Fard. Is there any weather alert today?"

for server in mcp_servers:
    await server.connect()
with trace("WeatherInfo_MCP_test"):
    result = await Runner.run(agent, question, max_turns=10)
display(Markdown(result.final_output))


There is currently a High Wind Warning in effect for Boston, Massachusetts. It is expected from 8 AM to 3 PM today with south winds of 25 to 35 mph and gusts up to 60 mph. The warning indicates damaging winds that can blow down trees and power lines, possibly causing power outages. Travel may be difficult, especially for high-profile vehicles. Please take precautions such as staying indoors and avoiding driving if possible during this period.